This script was used to clean the data (e.g. removing duplicates) and to add additional information extracted from the data (e.g. text statistics).

In [1]:
import os
import pandas as pd
from nltk import sent_tokenize, word_tokenize
from tqdm.notebook import tqdm
# from ydata_profiling import ProfileReport
from string import punctuation
from global_variables import RANDOM_SEED, LANG
import re

# local files
from preprocessing import Vocabulary # just for manual inspection of the remaining words
from functions_small_helper import create_folder_if_not_exists

In [2]:
REPLACEMENT_TOKS = ["[padnr]", "[namn]", "[pnr]", "[datum]", "[decimal]", "[time]"]


1) Load the data

In [3]:
# input data
data_folder = os.path.join("..", "data", "data_raw")
data_filename = "data_raw_replaced_labs.csv"
data_file = os.path.join(data_folder, data_filename)


# output data
clean_data_folder = os.path.join("..", "data", "data_clean")
create_folder_if_not_exists(clean_data_folder)

data_cleaned = os.path.join(clean_data_folder, "data_clean_no_dup")
data_partial_duplicates = os.path.join(clean_data_folder, "data_clean_part_dup")

<..\241028_run_dataprep> exists:  True


In [4]:
df_raw = pd.read_csv(data_file, header=0, index_col=0, dtype=object, encoding="utf-8")
# add row to keep original index if needed for manual inspection
if not "orig_index" in df_raw.columns:
    df_raw.insert(loc=0, column="orig_index", value=list(df_raw.index))
df_raw.reset_index(inplace=True, drop=True)

df_raw.head(0)

,orig_index,lab,remark,date,icdo3code,icdo3text,snomed3code,snomed3text,text


In [5]:
print(df_raw.shape)
print(df_raw.info())

(34051, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34051 entries, 0 to 34050
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   orig_index   34051 non-null  int64 
 1   lab          34051 non-null  object
 2   remark       24002 non-null  object
 3   date         31077 non-null  object
 4   icdo3code    34051 non-null  object
 5   icdo3text    34051 non-null  object
 6   snomed3code  34039 non-null  object
 7   snomed3text  34039 non-null  object
 8   text         34051 non-null  object
dtypes: int64(1), object(8)
memory usage: 2.3+ MB
None


2) Remove full duplicates and rows where snomed (=morphology, or morphology+grade) codes are missing

In [6]:

# create new df that will be manipulated to remove full duplicates
df = df_raw.copy(deep=True)

print(f"Number of duplicated rows: ", df.iloc[:, 1:].duplicated().sum())    # exclude "orig_index" column since it will not be a duplicate
df = df[df.iloc[:, 1:].duplicated()==False]
assert df.iloc[:, 1:].duplicated().sum() == 0
df.reset_index(inplace=True, drop=True)
print(f"Original shape: {df_raw.shape} \n--> New shape: {df.shape}")


Number of duplicated rows:  472
Original shape: (34051, 9) 
--> New shape: (33579, 9)


In [7]:
print(f"Number of missing values:")
no_snomed_index = df[df["snomed3code"].isna()].index
df = df.drop(no_snomed_index,  inplace=False)
df.reset_index(inplace=True, drop=True)
print(f"Removed {len(no_snomed_index)} reports without snomed code.\n",
      "New shape: ", df.shape)
df.tail(0)

Number of missing values:
Removed 12 reports without snomed code.
 New shape:  (33567, 9)


,orig_index,lab,remark,date,icdo3code,icdo3text,snomed3code,snomed3text,text


3) Replace missing dates and add year column

In [8]:
# replace date
df["date"] = df["date"].fillna("1900-01-01")

# add year column for eaiser access
def get_year(datestring):
    year, month, day = datestring.split("-")
    return int(year)

df["year"] = df["date"].apply(get_year)
df.tail(0)

,orig_index,lab,remark,date,icdo3code,icdo3text,snomed3code,snomed3text,text,year


4) Add site and subsite columns

In [9]:
def get_site(topo_code):
    assert len(topo_code) == 4  # C509
    return int(topo_code[1:-1]) # 50

def get_subsite(topo_code):
    assert len(topo_code) == 4
    return int(topo_code[-1])   # 9

df["site"] = df["icdo3code"].apply(get_site)
df["subsite"] = df["icdo3code"].apply(get_subsite)
df.sample(0)

,orig_index,lab,remark,date,icdo3code,icdo3text,snomed3code,snomed3text,text,year,site,subsite


5) Add site+subsite column (for accessing site directly in multitask prediction)


In [10]:
def get_site_subsite(topo_code):
    assert len(topo_code) == 4
    return int(topo_code[1:])

df["site+subsite"] = df["icdo3code"].apply(get_site_subsite)
df.head(0)

,orig_index,lab,remark,date,icdo3code,icdo3text,snomed3code,snomed3text,text,year,site,subsite,site+subsite


In [11]:
# split the code into subparts 12345(6) -> morphology: 1234, behavior: 5, grade: 6 (though only for few reports)
def get_hist(snomend_code):
    return snomend_code[0:4]

def get_behavior(snomed_code):
    if len(snomed_code) == 5:
        return snomed_code[-1]
    elif len(snomed_code) == 6:
        return snomed_code[-2]

def get_grade(snomed_code):
    if len(snomed_code) == 6:
        return snomed_code[-1]
    else:
        return "None"


def get_morph_no_grade(morph_code):
    return morph_code[:5]

df["histology"] = df["snomed3code"].apply(get_hist)
df["behavior"] = df["snomed3code"].apply(get_behavior)
df["morphology"] = df["snomed3code"].apply(get_morph_no_grade)
df["grade"] = df["snomed3code"].apply(get_grade)


In [12]:
print("Number of different full morphology codes (all (5 or 6) digits):", len(df["snomed3code"].unique()))
print("Number of different morphology codes (only 5 digit, no grade):", len(df["morphology"].unique()))
print("Number of different histology codes (first 4 digits):", len(df["histology"].unique()))
print("Number of different behavior codes (5th digit):", len(df["behavior"].unique()), df["behavior"].unique())
print("Number of different grade codes (6th digit):", len(df["grade"].unique()), df["grade"].unique())



Number of different full morphology codes (all (5 or 6) digits): 140
Number of different morphology codes (only 5 digit, no grade): 131
Number of different histology codes (first 4 digits): 99
Number of different behavior codes (5th digit): 4 ['3' '2' '1' '0']
Number of different grade codes (6th digit): 5 ['None' '2' '3' '1' '6']


6) Analyse text

In [13]:
# many texts have the same few words in the beginning, e.g "macroscopie : [padnr] [namn], [namn]+ [pnr]"
# one lab also has a date in the beginning; both of these are removed, to avoid classification based on these patterns

def remove_prefix(text):
    # remove all combinations of the replacement tokens [padnr], [namn], [pnr], and the punctuation marks , : and whitespace
    text = re.sub(pattern=r"macroscopie : ((\[padnr\])|(\[pnr\])|(\[namn\])|[,: \-])+", repl="", string=text, count=1)

    if text.startswith("dikt datum"):
        text = re.sub(pattern=r"dikt datum (20)?\d\d - \d\d - \d\d \. ", repl="", string=text, count=1)
    return text

# minimize vocabulary, similar to previous papers
def replace_date(text):
    # for dates that ocurr in the text;
    text = re.sub(pattern=r"(20)?\d\d - \d\d - \d\d", repl="[datum]", string=text, count=0) # count=0, all will be replaced
    return text
def replace_decimal(text):
    text = re.sub(pattern=r"(\d , \d cm)", repl="[decimal] cm", string=text, count=0) # count=0, all will be replaced
    text = re.sub(pattern=r"(\d , \d mm)", repl="[decimal] mm", string=text, count=0)
    return text

def replace_large_numbers(text):
    text = re.sub(pattern=r"\d\d\d+", repl="[large_number]", string=text, count=0)
    return text

def replace_time(text):
    text = re.sub(pattern=r"kl \d\d . \d\d", repl="[time]", string=text, count=0)
    return text

def replace_time_position(text):    # for breast cancer, position is given with clock reference
    text = re.sub(pattern=r"kl \. (\d\d?)", repl=r"kl \1", string=text, count=0)
    return text


In [14]:
# clean the text before tokenizing further
df["text"] = df["text"].apply(remove_prefix)
df["text"] = df["text"].apply(replace_date)
df["text"] = df["text"].apply(replace_decimal)
df["text"] = df["text"].apply(replace_time)
df["text"] = df["text"].apply(replace_time_position)
df["text"] = df["text"].apply(replace_large_numbers)

In [15]:
# tokenize with nltk tokenizer
# and undo nltk splits for replacement tokens
def merge_replacement_tokens(tokenized_text, replacement_tokens=REPLACEMENT_TOKS):
    # merge lists with dummy separator token
    sep_token = "<replacementtoken>"
    tmp_text = sep_token.join(tokenized_text)

    # merge the replacement tokens back together
    for repl_tok in replacement_tokens:
        repl_tok_no_bracket = repl_tok[1:-1]    # [namn] > namn to insert it in replace function
        
        tmp_text = tmp_text.replace(f"[{sep_token}{repl_tok_no_bracket}{sep_token}]", repl_tok)
    # delete the separator token again
    cleaned_tokenized_text = tmp_text.split(sep_token)
    return cleaned_tokenized_text


def get_tokenized_text(text):
    tokenized_text = word_tokenize(text, language=LANG)
    clean_text = merge_replacement_tokens(tokenized_text)

    assert type(clean_text) == list
    return clean_text


In [16]:
def get_senttokenized_text(text):
    sent_tokenized_text = sent_tokenize(text, language=LANG)
    cleaned_sents = []
    for sent in sent_tokenized_text:
        if len(cleaned_sents) == 0:     # just for first time
            cleaned_sents.append(sent)
            continue
        if re.match(pattern=r"(\d+|i+|i+v|vi+) \.$", string=cleaned_sents[-1]): 
            # if the previous sentence ends with a bulletpoint-like enumeration 
            # (with latin or roman numbers) followed by fullstop 
            # -> add what comes after to this (previous) bullet point)
            # otherwise, the sentence only consists of e.g. "1 ."
            # which is unwanted, esp. for HiSAN models

            cleaned_sents[-1] = cleaned_sents[-1] + " " + sent

    
        else:
            cleaned_sents.append(sent)
    # custom tokenize function is used here again
    word_sent_tokenized_sents = [get_tokenized_text(sent) for sent in cleaned_sents]

    return word_sent_tokenized_sents

def get_longest_sent_per_text(wordsent_tokenized_text):
    lengths = [len(sent) for sent in wordsent_tokenized_text]
    return max(lengths)


df["sent_tokenized_text"] = df["text"].apply(get_senttokenized_text)
df["longest_sent"] = df["sent_tokenized_text"].apply(get_longest_sent_per_text)

In [17]:
def split_long_sents(senttokenized_text, max_len=50):
    chunked_sents = []
    for sentence in senttokenized_text:
        while len(sentence) > max_len:
            chunked_sents.append(sentence[:max_len])
            sentence = sentence[max_len:]
        if len(sentence) > 0:
            chunked_sents.append(sentence)
    assert max([len(sent) for sent in chunked_sents]) <= max_len
    return chunked_sents

In [18]:
def get_n_sents(senttokenized_text):
    return len(senttokenized_text)


In [19]:
def get_avg_chunk_lengths(chunked_sents):
    lengths = [len(chunked_sent) for chunked_sent in chunked_sents]
    n_chunks = len(chunked_sents)

    return round(sum(lengths) / n_chunks, ndigits=3)

In [20]:
df["chunked_sent_tokenized_text"] = df["sent_tokenized_text"].apply(split_long_sents)
df["n_chunked_sents"] = df["chunked_sent_tokenized_text"].apply(get_n_sents)
df["avg_chunk_length"] = df["chunked_sent_tokenized_text"].apply(get_avg_chunk_lengths)

In [21]:
def get_n_tokens(tokenized_text):
    return len(tokenized_text)

def get_n_types(tokenized_text):
    return len(set(tokenized_text))



In [22]:
def is_valid_token(token, replacement_tokens=REPLACEMENT_TOKS):
    if token.isalpha():
        return True
    elif token in replacement_tokens:
        return True
    elif token not in punctuation:
        return True
    else:
        return False
    
def get_n_alphab_tokens(tokenized_text):
    return [is_valid_token(token) for token in tokenized_text].count(True)

def get_n_alphab_types(tokenized_text):
    return [is_valid_token(token) for token in set(tokenized_text)].count(True)



In [23]:
df["tokenized_text"] = df["text"].apply(get_tokenized_text)

In [24]:
## add more columns for text-related statisitcs

df["n_sents"] = df["sent_tokenized_text"].apply(get_n_sents)

df["n_tokens"] = df["tokenized_text"].apply(get_n_tokens)
df["n_types"] = df["tokenized_text"].apply(get_n_types)


df["n_alphab_tokens"] = df["tokenized_text"].apply(get_n_alphab_tokens)
df["n_alphab_types"] = df["tokenized_text"].apply(get_n_alphab_types)

df["avg_tokens_per_sent"] = round(df["n_tokens"] / df["n_sents"], ndigits=3)
df["avg_alphab_tokens_per_sent"] = round(df["n_alphab_tokens"] / df["n_sents"], ndigits=3)



In [25]:
# there were about 200 texts that only consisted of ". . ", these are removed, too
print("Old shape: ", df.shape)
no_alphab_tokens = df[df["n_alphab_tokens"] == 0]["text"].index
df = df.drop(no_alphab_tokens,  inplace=False)
df.reset_index(inplace=True, drop=True)
print("New shape: ", df.shape)
df.tail(0)


Old shape:  (33567, 30)
New shape:  (33363, 30)


,orig_index,lab,remark,date,icdo3code,icdo3text,snomed3code,snomed3text,text,year,...,n_chunked_sents,avg_chunk_length,tokenized_text,n_sents,n_tokens,n_types,n_alphab_tokens,n_alphab_types,avg_tokens_per_sent,avg_alphab_tokens_per_sent


7) Extract text duplicates: full and partial duplicates
--> these will be removed from the dataset. Partial duplicates will still be saved, if needed for future projects.

In [26]:
# choice to remove all text duplicates  so model only sees one code per identical text
print(df[df.duplicated(subset=["text"])].shape)


(2128, 30)


In [27]:
text_duplicate_indices = df[df.duplicated("text")].index
print("Number of duplicated texts: ", len(text_duplicate_indices))
df["text"][text_duplicate_indices].to_csv(os.path.join(clean_data_folder, "tmp_inspect_dupliacted_texts.csv"))

Number of duplicated texts:  2128


In [28]:
print("Old shape: ", df.shape)
df_no_full_text_duplicates = df.drop_duplicates("text", keep="first", inplace=False, ignore_index=True)
print("New shape: ", df_no_full_text_duplicates.shape)

Old shape:  (33363, 30)
New shape:  (31235, 30)


In [29]:
# Remove rows where there is another row with an additional remark, i.e.
# text, text+remark, text+remark+remark; these texts are removed from the dataset

assert list(df_no_full_text_duplicates.index) == list(range((df_no_full_text_duplicates.shape[0])))
assert df_no_full_text_duplicates.index[0] == 0

unique_texts = []
partial_duplicate_texts = []
duplicate_texts = []    
# Note: duplicate_texts are not full duplicate rows, only the text is identical;
# hence, there should be none at this point, they were removed before, so this is just a sanity check
for i, ind in enumerate(df_no_full_text_duplicates.index[:-1]):                                     # -1 to prevent index error for last text
    if i == 0: 
        text = df_no_full_text_duplicates["text"][ind]
        next_text = df_no_full_text_duplicates["text"][df_no_full_text_duplicates.index[i+1]]
        if text == next_text:
            duplicate_texts.append(ind)
        elif next_text.startswith(text) \
            and len(text) < len(next_text):    
            # if the text is fully contained in the next text, then move the text to the duplicate list
            partial_duplicate_texts.append(ind)
        else:
            unique_texts.append(ind)
    else:
        text = df_no_full_text_duplicates["text"][ind]
        next_text = df_no_full_text_duplicates["text"][df_no_full_text_duplicates.index[i+1]]
        previous_text = df_no_full_text_duplicates["text"][df_no_full_text_duplicates.index[i-1]]
        if text == next_text:
            duplicate_texts.append(ind)
        elif next_text.startswith(text) \
            and len(text) < len(next_text):    
            # if the text is partially contained in the next text, then move the text to the half duplicate list
            partial_duplicate_texts.append(ind)
        
        elif text.startswith(previous_text) \
            and len(previous_text) < len(text):
            partial_duplicate_texts.append(ind)
        else:
            unique_texts.append(ind)

if ind not in partial_duplicate_texts and ind not in duplicate_texts:
    unique_texts.append(df_no_full_text_duplicates.index[-1])   # last text is always kept since duplicates would have been removed already

partial_duplicate_texts = list(set(partial_duplicate_texts))    # remove indices that were added multiple times (if any)


print(f"Unique texts: {len(unique_texts)}", f"Texts that are contained in other texts, too: {len(partial_duplicate_texts)}", f"Full Duplicates: {len(duplicate_texts)}", sep="\n")
assert len(unique_texts) + len(partial_duplicate_texts) + len(duplicate_texts) == len(df_no_full_text_duplicates["text"])
print(len(unique_texts) + len(partial_duplicate_texts) + len(duplicate_texts), "==", len(df_no_full_text_duplicates["text"]))

df_no_full_text_duplicates["text"][partial_duplicate_texts].to_csv(os.path.join(clean_data_folder, "tmp_inspect_partial_dupliacted_texts.csv"))


Unique texts: 29334
Texts that are contained in other texts, too: 1901
Full Duplicates: 0
31235 == 31235


In [30]:
df_no_text_duplicates = df_no_full_text_duplicates.loc[unique_texts]
assert len(df_no_text_duplicates["text"]) == len(set(df_no_text_duplicates["text"]))    # only unique texts


df_partial_duplicates = df_no_full_text_duplicates.loc[partial_duplicate_texts]


print(df_no_text_duplicates.shape)
print(df_partial_duplicates.shape)

(29334, 30)
(1901, 30)


8) Save dataframes

as pkl-files to keep their datatypes as they are; esp. important for the lists of tokenized text

In [31]:
# save final dataframes

# for computing (since data types are preserved in pkl file)
df_no_text_duplicates.to_pickle(f"{data_cleaned}.pkl")
df_partial_duplicates.to_pickle(f"{data_partial_duplicates}.pkl")

# for easier manual inspection
df_no_text_duplicates.to_csv(f"{data_cleaned}.csv", encoding="utf-8")
df_partial_duplicates.to_csv(f"{data_partial_duplicates}.csv", encoding="utf-8")

9) Investigate Vocabulary 

In [32]:
df_no_text_duplicates_voc = Vocabulary(df_no_text_duplicates["tokenized_text"])

In [33]:
df_no_text_duplicates_voc.vocabsize(min_count=1)
# Note: For model training, words with low frequencies are removed, 
# so vocabulary size will be smaller in model training

39579

In [34]:
frq_counts_df_no_text_duplicates = pd.DataFrame.from_dict(dict(df_no_text_duplicates_voc.word2frq), 
                                                          orient="index", columns=["frq"])
frq_counts_df_no_text_duplicates = frq_counts_df_no_text_duplicates.sort_values("frq", ascending=False)

frq_counts_df_no_text_duplicates.to_csv(os.path.join(clean_data_folder, 
                                                     "vocabulary_data_clean_no_dup.csv"), encoding="utf-8")

In [35]:
len(df_no_text_duplicates_voc.numbers)

110

In [36]:
len(df_no_text_duplicates_voc.single_letter)

70

In [37]:
len(df_no_text_duplicates_voc.single_letter)

70

In [38]:
len(df_no_text_duplicates_voc.punctuation)

22

In [39]:
df_no_text_duplicates["longest_sent"].describe()

count    29334.000000
mean        33.163394
std         32.867342
min          1.000000
25%         18.000000
50%         25.000000
75%         36.000000
max        745.000000
Name: longest_sent, dtype: float64

(For data protection, the files below are not made available)

In [40]:
df_no_text_duplicates["tokenized_text"].to_csv(os.path.join(clean_data_folder,"tmp_maunally_inspect_tokenized_text.csv"))

In [41]:
df_no_text_duplicates["text"].to_csv(os.path.join(clean_data_folder,"tmp_maunally_inspect_text.csv"))

In [42]:
df_no_text_duplicates["sent_tokenized_text"].to_csv(os.path.join(clean_data_folder,"tmp_maunally_inspect_sent_tokenized_text.csv"))